# Modelling using the Infinite Mixture Model

In many applications it is desirable to allow the model to adjust its complexity according to the complexity of the data. Consider for example the task of grouping objects into clusters / groups. This task often involves the specification of the number of groups. However, in real world applications it is often not known a prior how many groups are to be expected. Further the number of examples per group often is heavy tailed, making it necessary to allow the model to form new groups when new data points are observed.

A natural approach for such applications is the use of non-parametrics models. This tutorial will introduce how to use the Dirichlet process to model a mixture of infinitely many groups in Turing.

### Chinese Restaurant Process